<a href="https://colab.research.google.com/github/kaustubhpatil2611/ai_assignments/blob/master/Movie_Recommendation_recall_precision_fscore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Information Retrievel System- Movie Recommendation system**

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [111]:
import pandas as pd

movies = pd.read_csv("/content/drive/My Drive/MovieLens Datset/movies.csv",encoding="Latin1")
Ratings = pd.read_csv("/content/drive/My Drive/MovieLens Datset/ratings.csv")
Tags = pd.read_csv("/content/drive/My Drive/MovieLens Datset/tags.csv",encoding="Latin1")

Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']#normalizing data
Rating_avg.head()

,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,12882,1,4.0,1147195252,4.061321,-0.061321
1,12882,32,3.5,1147195307,4.061321,-0.561321
2,12882,47,5.0,1147195343,4.061321,0.938679
3,12882,50,5.0,1147185499,4.061321,0.938679
4,12882,110,4.5,1147195239,4.061321,0.438679


In [112]:
Ratings[Ratings['userId']==12882].head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [113]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [114]:
Ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [115]:
Tags.head()

,movieId,userId,tag,timestamp
0,3916,12882,sports,1147195545
1,4085,12882,Eddie Murphy,1147195966
2,33660,12882,boxing,1147195514
3,1197,320,must show,1145964801
4,1396,320,must show,1145964810


In [116]:
check = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='movieId')
check.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,2.5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN,3.5,NaN,NaN,NaN,NaN,2.0,NaN,3.0,4.0,NaN,3.0,NaN,NaN,4.0,NaN,1.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,4.5,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,4.0,NaN,NaN,4.0,2.5,NaN,NaN,3.5,NaN,NaN,NaN,NaN,2.5,4.5,1.0,NaN,NaN,4.0,4.0,NaN,NaN,3.5,4.0,NaN,4.0,4.5,1.5,NaN,NaN,2.5,3.0,4.5,4.0,NaN,NaN,3.0,4.5,3.5,3.0
910,5.0,4.0,3.5,NaN,3.5,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,2.0,NaN,4.5,4.5,NaN,NaN,4.0,NaN,NaN,3.5,NaN,4.0,NaN,NaN,NaN,NaN,3.5,3.5,2.0,3.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,3.5,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN


In [117]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')
final.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,NaN,NaN,NaN,NaN,NaN,-1.329457,NaN,-0.829457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.329457,-1.329457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,1.314526,NaN,NaN,NaN,NaN,1.314526,NaN,NaN,0.314526,0.314526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.685474,NaN,NaN,0.314526,NaN,-0.185474,NaN,NaN,NaN,NaN,-1.685474,NaN,-0.685474,0.314526,NaN,-0.685474,NaN,NaN,0.314526,NaN,-2.185474,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,0.705596,0.205596,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.705596,NaN,NaN,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.794404,0.205596,NaN,NaN,0.205596,-1.294404,NaN,NaN,-0.294404,NaN,NaN,NaN,NaN,-1.294404,0.705596,-2.794404,NaN,NaN,0.205596,0.205596,NaN,NaN,-0.294404,0.205596,NaN,0.205596,0.705596,-2.294404,NaN,NaN,-1.294404,-0.794404,0.705596,0.205596,NaN,NaN,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.39808,NaN,-0.39808,-0.398080,NaN,NaN,NaN,0.101920,NaN,NaN,NaN,0.10192,0.10192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.898080,NaN,NaN,NaN,NaN,NaN,0.101920,-0.898080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.398080,-1.898080,NaN,0.60192,0.601920,NaN,NaN,0.10192,NaN,NaN,-0.39808,NaN,0.10192,NaN,NaN,NaN,NaN,-0.39808,-0.398080,-1.898080,-0.89808,NaN,NaN,0.101920,NaN,-0.898080,NaN,NaN,-0.39808,NaN,NaN,NaN,-0.398080,NaN,NaN,NaN,NaN,0.601920,NaN,NaN


In [118]:
# Replacing NaN by Movie Average
final_movie = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [119]:
final_movie.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,-0.436518,-0.468109,-0.770223,-0.615331,0.320415,-1.329457,-0.690175,-0.829457,-0.094277,-0.818052,-0.249107,-0.106853,-0.682703,0.378911,0.340013,-0.111331,-0.771965,-0.773505,-0.09978,-0.248662,-0.342241,-0.588567,0.159313,-0.075,0.06939,0.607793,0.307477,-0.252657,-0.329457,-1.329457,0.293767,0.186124,-0.27799,0.172255,-0.110619,-0.381846,-0.777004,0.005915,-0.433017,...,-0.109443,0.204025,-0.004392,0.352636,0.370785,0.322406,0.302189,0.08928,0.079512,0.200612,0.136546,-0.028006,0.164753,0.498174,0.175075,-0.309899,0.342812,-0.077913,0.225157,0.106557,0.316606,0.120372,0.336436,0.249364,0.172288,0.022489,0.440565,-0.078989,0.10413,0.259815,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
320,0.200220,-0.436518,-0.468109,-0.770223,-0.615331,0.320415,-0.203889,-0.690175,-0.150642,-0.094277,-0.818052,-0.249107,-0.106853,-0.682703,0.378911,0.340013,-0.111331,-0.771965,-0.773505,-0.09978,-0.248662,-0.342241,-0.588567,0.159313,-0.075,0.06939,0.607793,0.307477,-0.252657,0.388694,-0.200784,0.293767,0.186124,-0.27799,0.172255,-0.110619,-0.381846,-0.777004,0.005915,-0.433017,...,-0.109443,0.204025,-0.004392,0.352636,0.370785,0.322406,0.302189,0.08928,0.079512,0.200612,0.136546,-0.028006,0.164753,0.498174,0.175075,-0.309899,0.342812,-0.077913,0.225157,0.106557,0.316606,0.120372,0.336436,0.249364,0.172288,0.022489,0.440565,-0.078989,0.10413,0.259815,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
359,1.314526,-0.436518,-0.468109,-0.770223,-0.615331,1.314526,-0.203889,-0.690175,0.314526,0.314526,-0.818052,-0.249107,-0.106853,-0.682703,0.378911,0.340013,-0.111331,-0.685474,-0.773505,-0.09978,0.314526,-0.342241,-0.185474,0.159313,-0.075,0.06939,0.607793,-1.685474,-0.252657,-0.685474,0.314526,0.293767,-0.685474,-0.27799,0.172255,0.314526,-0.381846,-2.185474,0.005915,-0.433017,...,-0.109443,0.204025,-0.004392,0.352636,0.370785,0.322406,0.302189,0.08928,0.079512,0.200612,0.136546,-0.028006,0.164753,0.498174,0.175075,-0.309899,0.342812,-0.077913,0.225157,0.106557,0.316606,0.120372,0.336436,0.249364,0.172288,0.022489,0.440565,-0.078989,0.10413,0.259815,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
370,0.705596,0.205596,-0.468109,-0.770223,-0.615331,1.205596,-0.203889,-0.690175,-0.150642,-0.094277,-0.818052,-0.249107,-0.106853,-0.682703,0.378911,0.340013,-0.111331,-0.771965,-0.773505,-0.09978,-0.248662,-0.342241,-0.588567,0.705596,-0.075,0.06939,0.607793,0.307477,-0.252657,1.205596,-0.200784,0.293767,0.186124,-0.27799,0.172255,-0.110619,-0.381846,-0.777004,0.005915,-0.433017,...,-0.794404,0.205596,-0.004392,0.352636,0.205596,-1.294404,0.302189,0.08928,-0.294404,0.200612,0.136546,-0.028006,0.164753,-1.294404,0.705596,-2.794404,0.342812,-0.077913,0.205596,0.205596,0.316606,0.120372,-0.294404,0.205596,0.172288,0.205596,0.705596,-2.294404,0.10413,0.259815,-1.294404,-0.794404,0.705596,0.205596,0.429868,0.306567,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.398080,-0.770223,-0.398080,-0.398080,-0.203889,-0.690175,-0.150642,0.101920,-0.818052,-0.249107,-0.106853,0.101920,0.101920,0.340013,-0.111331,-0.771965,-0.773505,-0.09978,-0.248662,-0.342241,-0.588567,-0.898080,-0.075,0.06939,0.607793,0.307477,-0.252657,0.101920,-0.898080,0.293767,0.186124,-0.27799,0.172255,-0.110619,-0.381846,-0.777004,0.005915,-0.433017,...,-0.398080,-1.898080,-0.004392,0.601920,0.601920,0.322406,0.302189,0.10192,0.079512,0.200612,-0.398080,-0.028006,0.101920,0.4981

In [120]:
final_user.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,-8.294574e-01,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,-1.329457e+00,1.893404e-16,-8.294574e-01,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,-3.294574e-01,-1.329457e+00,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,...,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16,1.893404e-16
320,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,...,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17,4.297638e-17
359,1.314526e+00,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,1.314526e+00,-1.135546e-16,-1.135546e-16,3.145258e-01,3.145258e-01,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-6.854742e-01,-1.135546e-16,-1.135546e-16,3.145258e-01,-1.135546e-16,-1.854742e-01,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.685474e+00,-1.135546e-16,-6.854742e-01,3.145258e-01,-1.135546e-16,-6.854742e-01,-1.135546e-16,-1.135546e-16,3.145258e-01,-1.135546e-16,-2.185474e+00,-1.135546e-16,-1.135546e-16,...,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16,-1.135546e-16
370,7.055961e-01,2.055961e-01,1.958963e-15,1.958963e-15,1.958963e-15,1.205596e+00,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,1.958963e-15,7.055961e-

In [121]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.060063,0.072075,0.043266,0.039305,0.045616,0.035341,0.038068,-1.248514e-02,0.050183,0.054092,0.033465,0.022670,0.126154,0.040799,0.037185,0.057644,0.025122,0.099331,0.074787,-0.014068,0.085619,0.025612,-0.077753,0.031668,0.023530,0.035985,0.028395,0.050490,0.061638,0.042351,0.050607,0.085542,0.064005,0.150968,0.089748,0.016363,0.152072,-0.024010,0.084487,...,0.098051,0.150762,0.044621,0.059883,0.025779,-0.012247,0.073318,0.115571,-0.003164,-0.029347,-0.028104,-0.014781,0.079841,0.086719,0.058335,0.030147,0.074533,0.042013,0.057231,0.062579,0.054507,0.019707,0.101334,0.019195,0.083581,0.081471,0.022737,0.096231,0.086519,-2.107738e-30,0.052632,0.104864,0.011358,0.029674,0.092552,0.017876,0.051371,0.077377,0.026924,-0.022727
320,0.060063,0.000000,0.063054,0.027315,0.006811,0.075620,0.011910,0.042509,8.859278e-32,0.067389,0.094096,0.027075,0.081181,0.112769,0.115683,-0.019041,0.067281,0.023103,0.178612,0.069101,0.030826,0.047998,0.054826,0.048060,-0.017636,0.035096,0.130301,0.098850,0.093140,0.175358,0.112067,0.045418,0.040291,0.113322,0.114264,0.078094,0.015466,0.143908,0.103990,0.097358,...,0.139018,0.099389,0.076827,0.086716,0.084780,0.041092,-0.020653,0.059496,0.039312,0.006270,0.052847,0.026312,0.057807,0.119811,0.036268,0.015588,0.080673,0.074018,0.159119,0.065811,0.075163,0.197053,0.122757,-0.007312,0.119167,0.084930,0.123965,0.148860,0.046513,-3.127799e-03,0.115325,0.065130,0.071996,0.097554,0.064769,-0.006251,0.077256,0.098845,0.038752,0.056639
359,0.072075,0.063054,0.000000,0.135836,0.076131,0.036757,0.046418,0.066544,4.287659e-02,0.109726,0.103606,0.074078,0.083495,0.108544,0.105273,0.090336,0.171797,0.042173,0.097642,0.063185,0.033466,0.091255,0.022727,0.016863,0.039266,0.071202,0.119431,0.054832,0.094266,0.059529,0.067309,0.108330,0.011763,0.087724,0.105287,0.084674,0.039859,0.082505,-0.023744,0.155337,...,0.093810,0.019195,0.069904,0.082246,0.030767,0.009669,-0.020447,0.029569,-0.031501,0.024651,0.054934,0.086611,0.064936,0.157955,0.048168,0.068332,0.074654,0.121211,0.096491,0.045586,0.061558,0.048304,0.056335,0.075844,0.112839,0.059213,0.085524,0.031384,0.086432,-1.150907e-02,0.120191,0.020672,0.032166,0.039599,0.108502,0.026371,0.075492,0.102698,0.099307,0.003147
370,0.043266,0.027315,0.135836,0.000000,0.108404,0.071655,0.070893,-0.003139,5.223516e-02,0.090241,0.130281,0.045340,0.024994,0.013804,0.100884,0.056301,0.136781,0.026465,0.076204,0.133776,0.059315,0.001900,0.019183,0.045782,0.027932,0.041196,0.098064,0.050647,0.075710,0.046873,0.047289,0.053318,0.061057,0.089613,0.137955,0.127379,0.030898,0.051115,0.022289,0.088772,...,0.097390,0.026579,0.130174,0.061356,0.028953,-0.030325,0.015702,0.051406,0.037940,0.032215,-0.135878,0.051708,0.031644,0.058526,-0.007105,0.079231,0.057712,0.037232,0.129099,0.029618,0.075959,0.049326,0.106989,0.015734,0.073179,0.015463,0.115510,0.050461,0.056026,9.514939e-03,0.091218,0.049594,0.004344,0.040692,0.110434,0.019767,-0.001364,0.052187,0.050997,0.009950
910,0.039305,0.006811,0.076131,0.108404,0.000000,0.021814,0.027339,-0.032211,-6.301121e-03,-0.007491,0.013776,0.035724,0.019892,0.025552,-0.003860,0.021981,0.030525,0.050039,0.047871,0.036584,0.019700,-0.019518,-0.024508,-0.024842,-0.035802,0.085419,0.088349,0.059115,0.026837,0.016708,0.044664,-0.034211,-0.012314,0.042145,0.111471,0.072073,0.052843,0.024620,-0.009871,0.065827,...,0.037647,0.044465,0.017315,0.066150,0.024728,-0.003301,0.001534,-0.007960,

In [122]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.921169,0.665659,0.673486,0.694247,0.894969,0.805780,0.851492,0.945224,0.705491,0.874074,0.637915,0.843165,0.754374,0.830154,0.886110,0.860780,0.800477,0.749784,0.640139,0.805567,0.864683,0.897571,0.800168,0.828677,0.787479,0.689374,0.786913,0.870659,0.843057,0.672456,0.944350,0.780760,0.725339,0.907715,0.854996,0.852003,0.751158,0.918703,0.814916,...,0.728479,0.932834,0.667260,0.603647,0.886717,0.803920,0.843698,0.937589,0.803829,0.738573,0.900540,0.553610,0.596385,0.725313,0.928449,0.785363,0.918364,0.676499,0.769696,0.806471,0.871967,0.836396,0.860036,0.682528,0.741846,0.729389,0.722865,0.939506,0.844365,0.862244,0.827564,0.895641,0.879290,0.916856,0.912146,0.922262,0.587738,0.671783,0.949138,0.740220
320,0.921169,0.000000,0.687225,0.691158,0.699527,0.916020,0.816931,0.874283,0.970234,0.724147,0.898785,0.657858,0.874958,0.765808,0.852743,0.903417,0.880063,0.821121,0.774021,0.646124,0.832984,0.875296,0.914425,0.838968,0.843038,0.809264,0.718787,0.813400,0.886342,0.879166,0.699191,0.968571,0.776015,0.752814,0.927416,0.876819,0.876651,0.760602,0.951609,0.835669,...,0.747433,0.951359,0.684130,0.626665,0.913350,0.829876,0.854618,0.957911,0.835632,0.762864,0.930084,0.577268,0.603729,0.749379,0.951663,0.799897,0.942030,0.698417,0.803273,0.822594,0.892297,0.871373,0.878538,0.695042,0.762763,0.744302,0.761116,0.964076,0.856482,0.885070,0.861798,0.909376,0.907009,0.938964,0.929049,0.943265,0.612746,0.695382,0.973853,0.768459
359,0.665659,0.687225,0.000000,0.534369,0.523475,0.655225,0.602806,0.629143,0.705042,0.542504,0.666958,0.479666,0.629049,0.571102,0.599584,0.664162,0.663180,0.590453,0.542068,0.467545,0.611702,0.645856,0.667849,0.593107,0.624436,0.593592,0.514661,0.577194,0.638693,0.618612,0.508098,0.704499,0.552404,0.523785,0.674971,0.641404,0.643912,0.549777,0.677114,0.621314,...,0.541142,0.686440,0.491036,0.455515,0.654571,0.576378,0.614372,0.692617,0.577754,0.559018,0.679888,0.433964,0.448495,0.565119,0.693764,0.592826,0.689967,0.503179,0.568523,0.598191,0.635046,0.620303,0.628176,0.516545,0.554225,0.519014,0.536480,0.696413,0.626938,0.639643,0.621820,0.654320,0.655839,0.679696,0.683900,0.686193,0.418283,0.489595,0.707370,0.534065
370,0.673486,0.691158,0.534369,0.000000,0.547560,0.671810,0.618456,0.628825,0.712683,0.548592,0.681880,0.476377,0.631095,0.544897,0.636603,0.667123,0.662527,0.593674,0.550052,0.512877,0.612584,0.629576,0.672038,0.613448,0.620697,0.589770,0.522460,0.579581,0.647919,0.622410,0.496681,0.709255,0.586660,0.550123,0.686695,0.659343,0.648442,0.538235,0.688082,0.623061,...,0.557678,0.694474,0.535038,0.451116,0.661503,0.558130,0.629735,0.704914,0.614044,0.557226,0.646940,0.420464,0.427409,0.535213,0.692374,0.616166,0.693180,0.486360,0.606993,0.588988,0.649742,0.633157,0.647244,0.494308,0.556506,0.517727,0.557726,0.706352,0.616129,0.653867,0.636688,0.673489,0.651209,0.688647,0.689265,0.692595,0.405881,0.497332,0.714011,0.546637
910,0.694247,0.699527,0.523475,0.547560,0.000000,0.680701,0.621463,0.634921,0.723574,0.528281,0.669111,0.491774,0.646721,0.561251,0.618959,0.678199,0.660600,0.614004,0.562544,0.479716,0.634552,0.650609,0.670620,0.608203,0.618083,0.621134,0.527832,0.611560,0.656335,0.638442,0.523770,0.718310,0.565680,0.548172,0.711097,0.667012,0.669245,0.545710,0.703564,0.628957,...,0.537823,0.715856,0.497409,0.476938,0.679129,0.610796,0.637877,0.712134,0.627731,0.577508,0.693146,0.462238,0.444515,0.540627,0.711775,0.612850,0.7

In [123]:
def find_n_neighbours(df,n):#finding neighbors for clustering
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [124]:
# top 30 neighbours for each user on basis of user similarity matrix
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,113673,117918,9050,12882,38187,102668,98880,43829,13215,78501,6988,5611,131835,86783,98781,94883,61305,59269,117861,128236,88608,120782,74472,53834,88928,42245,58265,89527,49830,63902
320,12288,113673,28159,79846,134627,112948,120729,97163,2945,4931,44400,61305,82880,21860,100540,12569,88608,124849,69256,59269,39271,94883,127683,101137,54989,134521,80946,10055,64365,106512
359,102118,96482,102532,50898,2702,60016,23428,120782,57937,42096,38159,32780,65670,124078,11343,46645,79531,35246,134181,128224,117258,7723,120729,61305,40768,117918,86768,129498,131620,58346
370,46645,42245,40768,23428,123707,60016,45120,113645,97195,102118,58265,113540,102532,120782,17039,117007,101137,57937,27365,41244,5611,20530,2702,38159,359,43354,117144,96482,2988,108195
910,87042,131620,67352,40768,31321,48821,26222,63295,5611,370,79531,84752,10164,17022,60016,133811,12271,88394,105455,35522,134521,88738,46645,108195,70201,58265,18115,114601,23428,17039


In [125]:
# top 30 neighbours for each user on basis of movie similarity matrix
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,138176,100240,96936,51460,88932,1447,104732,125012,5268,121403,104529,12930,13927,128079,78908,27142,42944,129869,51255,124454,121987,72633,21401,114335,22338,118304,124981,93203,81435,94333
320,138176,96936,121403,1447,51460,125012,88932,42944,5268,104529,12930,13927,129869,27142,51255,100240,128079,21401,114335,104732,121987,102549,118304,86309,94333,124981,93203,80585,136037,22338
359,138176,1447,5268,96936,100240,21401,88932,13927,104732,72633,121403,51255,42944,124981,128079,86309,65255,54114,93203,129869,12930,121987,114335,125012,51460,118304,57474,27142,80585,22338
370,86309,44194,138176,24802,129869,96936,1447,104529,94333,88932,51460,51255,12930,124454,5268,125012,104732,88455,54114,93203,124981,27142,102549,120308,54643,42944,80585,13927,21401,136037
910,96936,107991,138176,27142,51460,125012,88932,100240,72633,129869,1447,104732,121987,104529,124454,80070,12930,128079,13927,118304,36624,51255,94333,42944,121403,80585,61755,124981,88455,78908


In [126]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [140]:
def get_precision_recall_fscore(userID):
  seen=Ratings[Ratings['userId']==userID].sort_values(by=['rating'],ascending=False)['movieId']
  seen=list(seen)#getting list of movie IDS watched by the user
  nop=Ratings[Ratings['userId']==userID].sort_values(by=['rating'],ascending=False)
  nop=nop[nop['rating']>=3.5]#getting the movies which have been rated greater than 3.5 by user
  nop=nop[['movieId','rating']]
  good_movies=list(nop['movieId'])#list of good movies watched by user

  predicted=[]#list to score predicted scores of seen movies by user
  for movie in seen:
      score = User_item_score(userID,movie)#predict the likely movie score for the movies watched by user
      predicted.append(score)
  
  dp = pd.DataFrame(seen,columns=['movies']) #creating dataframe of movieIds and Score for easy processing
  dp['score']=predicted
  dp=dp[dp['score']>=3.5].sort_values(by=['score'],ascending=False)#getting the movies which have been predicted to be rated greater than 3.5
  recommended_movies=list(dp['movies'])#list of recommended movies
  
  good_movies_recommended=0
  for movie in recommended_movies:
    if movie in good_movies:#if movie predicted is good
      good_movies_recommended=good_movies_recommended+1#count of good movies recommended
  mov=list(movies['movieId'])
  title=list(movies['title'])
  print(" ")
  precision=good_movies_recommended/len(recommended_movies)#precision
  print("The precision of prediction is:",precision)
  recall=good_movies_recommended/len(good_movies)#recall
  print("The recall of prediction is:",recall)
  fscore=(2*precision*recall)/(precision+recall)#fscore
  print("The fscore of prediction is:",fscore)
  print(" ")
  print("Recommended Movies are: ")
  print(" ")
  for id in recommended_movies:#print recommended movies
    i=mov.index(id)
    print(title[i])

In [141]:
user = int(input("Enter the user id to whom you want to recommend : "))
get_precision_recall_fscore(user)

Enter the user id to whom you want to recommend : 370
 
The precision of prediction is: 0.7877094972067039
The recall of prediction is: 0.9368770764119602
The fscore of prediction is: 0.8558421851289834
 
Recommended Movies are: 
 
Godfather, The (1972)
Shawshank Redemption, The (1994)
Fight Club (1999)
Matrix, The (1999)
Casablanca (1942)
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Yojimbo (1961)
City of God (Cidade de Deus) (2002)
North by Northwest (1959)
Schindler's List (1993)
Memento (2000)
Pulp Fiction (1994)
Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
Usual Suspects, The (1995)
Lives of Others, The (Das leben der Anderen) (2006)
Lord of the Rings: The Fellowship of the Ring, The (2001)
Dogville (2003)
Great Dictator, The (1940)
Reservoir Dogs (1992)
Princess Bride, The (1987)
Night on Earth (1991)
Monty Python and the Holy Grail (1975)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Psycho (1960)
Intouchables (2011)
Ame